# Advent of Code 2025

In [ ]:
open FSharp.Collections
open System

let trueMod n x =
      x % n |> (function | rem when rem < 0  -> rem + n | rem -> rem)

let turn current (change: string) = 
    int (change.Substring(1))
    |> (if change[0] = 'R' then (+) else (-)) current 
    |> trueMod 100 

let part1 = Seq.scan turn 50 
    
let tick direction = function
    | _, 0                  -> None
    | current, remaining    ->
        if direction = 'R' then (current + 1) % 100 
        else trueMod 100 (current - 1 + 100)
        |> fun next -> Some (next, (next, remaining - 1))

let turns (head::rest) (change: string) = 
    List.unfold (tick change[0]) (head, int (change.Substring(1)))
    |> List.rev
    |> fun seq -> seq @ (head :: rest)

let part2 = Seq.fold turns [50]

let run filePath part =
    System.IO.File.ReadAllLines(filePath) 
    |> part
    |> Seq.sumBy (function | 0 -> 1 | _ -> 0)

run "day1.txt" part1
run "day1.txt" part2

## Day 2

In [ ]:
#r "nuget: FParsec, 1.1.1"
#r "nuget: FParsec-Pipes, 1.1.2"
#r "nuget: FSharpx.Collections, 3.1.0"

In [ ]:
open FParsec
open FParsec.Pipes
open FSharpx.Collections

let runParser parser input =
    match run parser input with
    | Success(result, _, _)     -> result
    | Failure(errorMsg, _, _)   -> failwith errorMsg

let inputParser = 
    let pRange = %% +.pint64 -- '-' -- +.pint64 -%> auto
    %% +.(qty[1..] / ',' * pRange) -%> ResizeArray.toSeq

let isInvalid (str: char array) = 
    Seq.length str |> fun l -> str[0..(l/2)-1] = str[l/2..l-1]

let findInvalids checker (a, b) =
    seq { for i in a..b do if checker (Seq.toArray (string i)) then yield i } 

let part1 = Seq.sumBy (findInvalids isInvalid >> Seq.sum)

let rec isInvalid2 (str: char array) = function
    | 0                                                                             -> false
    | shift when Array.permute (fun j -> (j + shift) % Seq.length str) str = str    -> true
    | shift                                                                         -> isInvalid2 str (shift - 1)

let part2 = 
    let wrapper chars = isInvalid2 chars (Seq.length chars / 2)
    Seq.sumBy (findInvalids wrapper >> Seq.sum)

let run filePath part =
    System.IO.File.ReadAllText(filePath) 
    |> runParser inputParser
    |> part

run "day2.txt" part1
run "day2.txt" part2
  

## Day 3

In [ ]:
let rec largest2 collected remaining = 
    match collected, remaining with
    | _         , []                                        -> collected
    | []        , head  :: next :: rest                     -> largest2 [head   ; next] rest
    | [fst; snd], head  :: rest          when snd  > fst    -> largest2 [snd    ; head] rest
    | [fst; _]  , head  :: next :: rest  when head > fst    -> largest2 [head   ; next] rest
    | [fst; snd], head  :: rest          when head > snd    -> largest2 [fst    ; head] rest
    | _         , _     :: rest                             -> largest2 collected       rest

let charToInt c = int c - int '0'

// For largest 12 digits

let (|CanShiftAtIndex|_|) (collected, ((head :: rest) as remaining)) =
    List.mapi (fun i x -> (i, x < head && List.length remaining >= (12 - i))) collected
    |> List.tryFind snd
    |> Option.map fst

let rec largest12 collected remaining =
    match collected, remaining with
    | _, []             -> collected |> List.take 12
    | [], _             -> largest12 [remaining[0]]                         remaining[1..]
    | CanShiftAtIndex i -> largest12 (collected[0..(i-1)] @ [remaining[0]]) remaining[1..]
    | _, head :: rest   -> largest12 (collected @ [head])                   rest

let run filePath (solver: int list -> int list -> int list)  =
    System.IO.File.ReadAllLines(filePath)
    |> Seq.sumBy (Seq.toList >> List.map charToInt >> solver [] >> String.Concat >> int64)

run "day3.txt" largest2 // part 1
run "day3.txt" largest12 // part 2


## Day 4

In [ ]:
let flip f a b = f b a

let parseInput (lines: string array) =
    seq { for i in 0 .. lines.Length - 1 do for j in 0 .. lines[i].Length - 1 do (i, j), lines[i][j] }
    |> Map.ofSeq

let bloom (map: Map<int * int, char>) (i, j) =
    [ (i - 1, j); (i + 1, j); (i, j - 1); (i, j + 1); (i - 1, j - 1); (i - 1, j + 1); (i + 1, j - 1); (i + 1, j + 1) ]
    |> List.filter (flip Map.tryFind map >> (=) (Some '@'))
    |> List.length

let findAccessibles map =
    Map.filter (fun (i, j) v -> v = '@' && bloom map (i, j) < 4) map
    |> Map.count

let findAccessibles2 map =
    Map.filter (fun (i, j) v -> not (v = '@' && bloom map (i, j) < 4)) map
    |> function
        | iteration when Map.count iteration = Map.count map    -> None
        | iteration                                             -> Some (Map.count map - Map.count iteration, iteration)

let run filePath solver = 
    System.IO.File.ReadAllLines(filePath)
    |> parseInput
    |> solver
     
run "day4.txt" findAccessibles // part 1
run "day4.txt" (Seq.unfold findAccessibles2 >> Seq.sum) // part 2

## Day 5

In [ ]:
let curry f a b = f (a, b)

let parse (input: string) =
    let rs :: ns :: _ = input.Split("\n\n") |> Array.toList
    let ranges = 
        rs.Split('\n')
        |> Seq.map (fun line -> 
            let parts = line.Split('-')
            int64 parts[0], int64 parts[1])
    let numbers = ns.Split('\n') |> Seq.map int64

    ranges, numbers

let isValid ranges n =
    Seq.exists (fun (low, high) -> n >= low && n <= high) ranges

let sieve (ranges, numbers) =
    numbers 
    |> Seq.filter (isValid ranges) 
    |> Seq.length

let tryMerge (l1, h1) (l2, h2) =
    if h1 < l2 || h2 < l1 
    then [ (l1, h1); (l2, h2) ] 
    else [min l1 l2, max h1 h2]

let rec rangeTraverser tested = function
    | []                        ,   nextRange -> nextRange :: tested
    | first :: restToCompare    ,   nextRange ->
        tryMerge first nextRange
        |> function
            | [merged] -> rangeTraverser tested             (restToCompare, merged)
            | [r1; r2] -> rangeTraverser (first :: tested)  (restToCompare, nextRange)

let sieve2 (ranges: (int64 * int64) list) =
    Seq.fold (fun rangesToCompare initialRange -> rangeTraverser [] (rangesToCompare, initialRange)) [ranges.Head] ranges.Tail
    |> Seq.sumBy (fun (low, high) -> high - low + 1L)

let run filePath solver =
    System.IO.File.ReadAllText(filePath) 
    |> parse 
    |> solver

run "day5.txt" sieve
run "day5.txt" (fst >> Seq.toList >> sieve2)